<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Logo_uninorte_colombia.jpg/1200px-Logo_uninorte_colombia.jpg" width=70%></img>

<font size=12>Algoritmos Y Complejidad</font>\
<font size=6>Algoritmos Geneticos: TSP</font>

Héctor Andrés Suárez Camacho - 200162159 \
Fernando Mateo Valencia Gómez - 200166289\
Santiago Carbal Martinez - 200152335\
Santiago Hernández Cadavid - 200153162

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import itertools


#Colombia_TSP 

Explicación de nuestra solución:


1.   Lo primero que hacemos es leer la matriz de distancia del archivo de excel.
2.   Luego obtenemos la lista de las ciudades de la primera fila del archivo.
3.   Después creamos la matriz de distancias para un acceso más fácil.
4.   Luego generamos unas coordenadas aleatorias para representar cada ciudad




In [ ]:
# Leer el archivo de Excel con la matriz de distancias
distances_df = pd.read_excel("colombia.xlsx", sheet_name="colombia")

# Obtener la lista de ciudades desde la primera fila
ciudades = distances_df.columns[:].tolist()

# Crear una matriz de distancias a partir del DataFrame
distances = distances_df.values

# Obtener las coordenadas aleatorias para cada ciudad
coordenadas = []
for _ in range(len(ciudades)):
    latitud = random.uniform(2, 13)  # Rango de latitudes para Colombia
    longitud = random.uniform(-80, -67)  # Rango de longitudes para Colombia
    coordenadas.append((latitud, longitud))



5.   Calculamos el fitness como la suma de todas las distancias por cada solución que se genera.
6. La población inicial para el algoritmo genético. Recibe el número de individuos en la población (num_individuos) y el número de nodos (ciudades) en el grafo (num_nodos). Genera individuos aleatorios, donde cada individuo es una lista de índices de ciudades permutados al azar.
7. Escogemos a las mejores soluciones por su fitness o aptitud, para asegurar que nuestra solución será mejor (para resolver el problema).
8. Luego hacemos el cruce o combinación, pero tomando dos puntos de corte para producir decendencia más variada.
9. La mutación opera intercambiando dos genes de posición, es decir, esocojo dos genes del cromosoma y los cambio de posición. (Se supone que cada cromosoma/solución no repite genes)
10. Retornamos la nueva generación



In [2]:
# Función para evaluar la aptitud (distancia total recorrida)
def evaluar_fitness(solucion):
    total_distance = 0
    for i in range(len(solucion) - 1):
        current_city = solucion[i]
        next_city = solucion[i + 1]
        total_distance += distances[current_city][next_city]
    return total_distance

# Generar población inicial
def generar_poblacion_inicial(num_individuos, num_nodos):
    poblacion = []
    for _ in range(num_individuos):
        individuo = random.sample(range(num_nodos), num_nodos)
        poblacion.append(individuo)
    return poblacion

# Generar la siguiente generación utilizando selección por torneo y operador de cruce de orden
def generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion):
    nueva_generacion = []
    
    # Realizar torneo para seleccionar padres
    for _ in range(num_padres):
        torneo = random.sample(poblacion, 5)
        mejor_individuo = min(torneo, key=lambda x: evaluar_fitness(x))
        nueva_generacion.append(mejor_individuo)
    
    # Operador de cruce de orden
    while len(nueva_generacion) < len(poblacion):
        padre1 = random.choice(nueva_generacion)
        padre2 = random.choice(nueva_generacion)
        
        punto_corte1 = random.randint(0, len(padre1) - 2)
        punto_corte2 = random.randint(punto_corte1 + 1, len(padre1) - 1)
        
        hijo = [-1] * len(padre1)
        hijo[punto_corte1:punto_corte2 + 1] = padre1[punto_corte1:punto_corte2 + 1]
        
        idx = punto_corte2 + 1
        for i in range(len(padre2)):
            if idx == len(padre2):
                idx = 0
            if padre2[i] not in hijo:
                hijo[idx] = padre2[i]
                idx += 1
        
        nueva_generacion.append(hijo)
    
    # Aplicar mutación
    for individuo in nueva_generacion:
        if random.random() < probabilidad_mutacion:
            idx1, idx2 = random.sample(range(len(individuo)), 2)
            individuo[idx1], individuo[idx2] = individuo[idx2], individuo[idx1]
    
    return nueva_generacion


11. Definimos los parametros del Algoritmo.
12. Luego generamos nuestra población inicial.
13. Luego iteramos tantas veces como generaciones queramos. En cada iteración calculamos el fitness de la población actual, luegos tomamos la solución que menor fitness tenga, debido a que loa distancia total de cada solución es su fitnees, una distancia menor es un menor fitness, y eso es lo que buscamos, que se demore menos. Una vez tengamos los mejores fitness, escogemos una nueva población, es decir, una generación.

In [ ]:
# Parámetros del algoritmo genético
num_generaciones = 1000
num_individuos = 27
num_padres = 10
probabilidad_mutacion = 0.05

# Obtener el número de nodos
num_nodos = len(ciudades)

# Generar población inicial
poblacion = generar_poblacion_inicial(num_individuos, num_nodos)

# Variables para guardar la evolución de la mejor distancia
mejores_distancias = []
mejor_solucion = None

# Ejecutar el algoritmo genético
for i in range(num_generaciones):
    # Calcular la distancia de la población actual
    distancias = [evaluar_fitness(individuo) for individuo in poblacion]
    
    # Encontrar y guardar la mejor solución actual
    mejor_solucion_actual = min(poblacion, key=lambda x: evaluar_fitness(x))
    if mejor_solucion is None or evaluar_fitness(mejor_solucion_actual) < evaluar_fitness(mejor_solucion):
        mejor_solucion = mejor_solucion_actual
    
    # Guardar la mejor distancia de la generación actual
    mejores_distancias.append(evaluar_fitness(mejor_solucion))
    
    # Generar la siguiente generación
    poblacion = generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion)

# Graficar el grafo completo
grafo = nx.Graph()
for ciudad in ciudades:
    x = coordenadas[ciudades.index(ciudad)][0]
    y = coordenadas[ciudades.index(ciudad)][1]
    grafo.add_node(ciudad, pos=(x, y))

for nodo1, nodo2 in itertools.combinations(ciudades, 2):
    grafo.add_edge(nodo1, nodo2, weight=distances[ciudades.index(nodo1)][ciudades.index(nodo2)])

pos = nx.get_node_attributes(grafo, 'pos')
labels = {nodo: nodo for nodo in grafo.nodes()}
nx.draw_networkx(grafo, pos, labels=labels, with_labels=True, node_color='skyblue', edge_color='gray')
plt.title('Grafo Completo')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.grid(True)
plt.show()

# Imprimir el recorrido del viajante de comercio
print("Mejor Recorrido:")
for i in range(len(mejor_solucion)):
    print(ciudades[mejor_solucion[i]], end=' -> ')
print(ciudades[mejor_solucion[0]])

# Imprimir su distancia
print("Distancia:", evaluar_fitness(mejor_solucion))

# Graficar el recorrido de la mejor solución
mejor_rec = mejor_solucion + [mejor_solucion[0]]  # Cerrar el ciclo
x = [coordenadas[nodo][0] for nodo in mejor_rec]
y = [coordenadas[nodo][1] for nodo in mejor_rec]
plt.figure(figsize=(8, 8))
plt.plot(x, y, 'bo-')
plt.title('Recorrido del Viajante de Comercio')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.show()

# Graficar fitness vs generación
generaciones = range(num_generaciones)
plt.plot(generaciones, mejores_distancias)
plt.title('Mejor Distancia por Generación')
plt.xlabel('Generación')
plt.ylabel('Mejor Distancia')
plt.show()

En la grafica de Mejor distancia vs generación, vemos que con el pasar las generaciones obtenemos recorridos que demoran menos.

#Ciudades_TSP



1.   Primero, como nada más tenemos las coordenadas de las ciudades, lo primero que hay que hacer es hallar las distancias entre todos los nodos. Para eso usamos la formula de la distancia euclidiana

2. Luego, tenemos la función evaluar-fitness donde por cada individuo (es decir, un recorrido) vamos sumando las distancias que hay en el.

3. Para escoger a los padres, hallamos las probabilidades de cada individuo frente a la población actual. Luego se escogen los padres en función de esas probabilidades. Un individuo puede ser seleccionado como padre varias veces.

4. En la función recombinar, se esocoge un punto de corte por parejas de padres y luego se combinan las partes de los padres para formar a los hijos.

5. La mutación funciona intercambiando la posición de dos genes de cada individuo.

6. La función generar_nueva_decendencia, combina las funciones anteriores para generar una nueva generación de individuos.

7. La función encontrar_mejor_solución lo que hace es buscar al individuo con menor distancia total de recorrido.



In [6]:
def distancia_euclidiana(nodo1, nodo2, coordenadas):
    x1, y1 = coordenadas[nodo1 * 2], coordenadas[nodo1 * 2 + 1]
    x2, y2 = coordenadas[nodo2 * 2], coordenadas[nodo2 * 2 + 1]
    distancia = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distancia

def evaluar_fitness(individuo, coordenadas):
    distancia = 0
    for i in range(len(individuo) - 1):
        nodo_actual = individuo[i]
        nodo_siguiente = individuo[i + 1]
        distancia += distancia_euclidiana(nodo_actual, nodo_siguiente, coordenadas)
    return distancia

def seleccionar_padres(poblacion, num_padres):
    padres = []
    fitness_total = sum(evaluar_fitness(individuo, coordenadas) for individuo in poblacion)
    probabilidades = [evaluar_fitness(individuo, coordenadas) / fitness_total for individuo in poblacion]
    for _ in range(num_padres):
        padre = random.choices(poblacion, weights=probabilidades)[0]
        padres.append(padre)
    return padres

def recombinar(padre1, padre2):
    punto_corte = random.randint(0, len(padre1) - 1)
    hijo1 = padre1[:punto_corte] + [nodo for nodo in padre2 if nodo not in padre1[:punto_corte]]
    hijo2 = padre2[:punto_corte] + [nodo for nodo in padre1 if nodo not in padre2[:punto_corte]]
    return hijo1, hijo2

def mutar(individuo, probabilidad_mutacion):
    if random.random() < probabilidad_mutacion:
        i, j = random.sample(range(len(individuo)), 2)
        individuo[i], individuo[j] = individuo[j], individuo[i]
    return individuo

def generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion, coordenadas):
    padres = seleccionar_padres(poblacion, num_padres)
    descendencia = []
    while len(descendencia) < len(poblacion):
        padre1, padre2 = random.sample(padres, 2)
        hijo1, hijo2 = recombinar(padre1, padre2)
        hijo1 = mutar(hijo1, probabilidad_mutacion)
        hijo2 = mutar(hijo2, probabilidad_mutacion)
        descendencia.extend([hijo1, hijo2])
    return descendencia[:len(poblacion)]

def encontrar_mejor_solucion(poblacion, coordenadas):
    mejor_distancia = float('inf')
    mejor_solucion = None
    for individuo in poblacion:
        distancia = evaluar_fitness(individuo, coordenadas)
        if distancia < mejor_distancia:
            mejor_distancia = distancia
            mejor_solucion = individuo
    return mejor_solucion

def leer_coordenadas(nombre_archivo):
    coordenadas = []
    with open(nombre_archivo, 'r') as archivo:
        lineas = archivo.readlines()
        for linea in lineas:
            coordenada = int(linea.strip())
            coordenadas.append(coordenada)
    return coordenadas

8. En primer lugar, se utiliza la función generar_poblacion_inicial para generar una población inicial de individuos. Cada individuo es una lista de nodos que representa un posible recorrido del viajante de comercio. Se generan num_individuos individuos de manera aleatoria utilizando la función random.sample para garantizar que no haya nodos repetidos en cada individuo.

9. Luego, se ejecuta el algoritmo genético en un bucle for que itera num_generaciones veces. En cada iteración, se calcula la distancia de cada individuo en la población actual utilizando la función evaluar_fitness, que suma las distancias entre nodos consecutivos en el recorrido. También se encuentra y guarda la mejor solución actual utilizando la función encontrar_mejor_solucion.

10. A continuación, se genera la siguiente generación de individuos utilizando la función generar_nueva_generacion. En esta función, se seleccionan los padres mediante el método de selección proporcional al fitness, se realiza la recombinación y la mutación para generar la descendencia.

11. Una vez finalizadas las iteraciones del bucle, se tienen los resultados finales. Se grafica el grafo completo que representa todas las ciudades, y se muestra el recorrido de la mejor solución encontrada en forma de grafo. También se calcula y muestra la distancia total de la mejor solución, así como el recorrido en forma de lista.

##Para 100 ciudades


In [16]:
# Leer coordenadas desde archivo
coordenadas = leer_coordenadas('map_city100.tsp')

# Parámetros del algoritmo genético
num_nodos = len(coordenadas) // 2
num_individuos = 100
num_generaciones = 10
num_padres = 100
probabilidad_mutacion = 0.1

In [ ]:
# Generar población inicial
def generar_poblacion_inicial(num_individuos, num_nodos):
    poblacion = []
    for _ in range(num_individuos):
        individuo = random.sample(range(num_nodos), num_nodos)
        poblacion.append(individuo)
    return poblacion

poblacion = generar_poblacion_inicial(num_individuos, num_nodos)

# Variables para guardar la evolución de la mejor distancia
mejores_distancias = []
mejor_solucion = None

# Ejecutar el algoritmo genético
for i in range(num_generaciones):
    # Calcular la distancia de la población actual
    distancias = [evaluar_fitness(individuo, coordenadas) for individuo in poblacion]
    
    # Encontrar y guardar la mejor solución actual
    mejor_solucion_actual = encontrar_mejor_solucion(poblacion, coordenadas)
    if mejor_solucion is None or evaluar_fitness(mejor_solucion_actual, coordenadas) < evaluar_fitness(mejor_solucion, coordenadas):
        mejor_solucion = mejor_solucion_actual
    
    # Guardar la mejor distancia de la generación actual
    mejores_distancias.append(evaluar_fitness(mejor_solucion, coordenadas))
    
    # Generar la siguiente generación
    poblacion = generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion, coordenadas)

# Graficar el grafo completo
grafo = nx.Graph()
for i in range(num_nodos):
    x = coordenadas[i * 2]
    y = coordenadas[i * 2 + 1]
    grafo.add_node(i, pos=(x, y))

for nodo1, nodo2 in itertools.combinations(range(num_nodos), 2):
    grafo.add_edge(nodo1, nodo2, weight=distancia_euclidiana(nodo1, nodo2, coordenadas))

pos = nx.get_node_attributes(grafo, 'pos')
labels = {nodo: str(nodo) for nodo in grafo.nodes()}
nx.draw_networkx(grafo, pos, labels=labels, with_labels=True, node_color='skyblue', edge_color='gray')
plt.title('Grafo Completo')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.grid(True)
plt.show()

# Calcular y mostrar la aptitud de la solución final
print("Distancia:", evaluar_fitness(mejor_solucion, coordenadas))
# Mostrar el recorrido en consola
print("Mejor recorrido:")
for nodo in mejor_solucion:
    print(nodo, end=' -> ')
print(mejor_solucion[0])

# Graficar el recorrido de la mejor solución
mejor_rec = mejor_solucion + [mejor_solucion[0]]  # Cerrar el ciclo
x = [coordenadas[nodo * 2] for nodo in mejor_rec]
y = [coordenadas[nodo * 2 + 1] for nodo in mejor_rec]
plt.figure(figsize=(8, 8))
plt.plot(x, y, 'bo-')
plt.title('Recorrido del Viajante de Comercio')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.show()

# Graficar fitness vs generación
generaciones = range(num_generaciones)
plt.plot(generaciones, mejores_distancias)
plt.title('Mejor Distancia por Generación')
plt.xlabel('Generación')
plt.ylabel('Mejor Distancia')
plt.show()

##Para 400 ciudades


In [7]:
# Leer coordenadas desde archivo
coordenadas = leer_coordenadas('map_city400.tsp')

# Parámetros del algoritmo genético
num_nodos = len(coordenadas) // 2
num_individuos = 400
num_generaciones = 10
num_padres = 100
probabilidad_mutacion = 0.1

In [ ]:
# Generar población inicial
def generar_poblacion_inicial(num_individuos, num_nodos):
    poblacion = []
    for _ in range(num_individuos):
        individuo = random.sample(range(num_nodos), num_nodos)
        poblacion.append(individuo)
    return poblacion

poblacion = generar_poblacion_inicial(num_individuos, num_nodos)

# Variables para guardar la evolución de la mejor distancia
mejores_distancias = []
mejor_solucion = None

# Ejecutar el algoritmo genético
for i in range(num_generaciones):
    # Calcular la distancia de la población actual
    distancias = [evaluar_fitness(individuo, coordenadas) for individuo in poblacion]
    
    # Encontrar y guardar la mejor solución actual
    mejor_solucion_actual = encontrar_mejor_solucion(poblacion, coordenadas)
    if mejor_solucion is None or evaluar_fitness(mejor_solucion_actual, coordenadas) < evaluar_fitness(mejor_solucion, coordenadas):
        mejor_solucion = mejor_solucion_actual
    
    # Guardar la mejor distancia de la generación actual
    mejores_distancias.append(evaluar_fitness(mejor_solucion, coordenadas))
    
    # Generar la siguiente generación
    poblacion = generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion, coordenadas)

# Graficar el grafo completo
grafo = nx.Graph()
for i in range(num_nodos):
    x = coordenadas[i * 2]
    y = coordenadas[i * 2 + 1]
    grafo.add_node(i, pos=(x, y))

for nodo1, nodo2 in itertools.combinations(range(num_nodos), 2):
    grafo.add_edge(nodo1, nodo2, weight=distancia_euclidiana(nodo1, nodo2, coordenadas))

pos = nx.get_node_attributes(grafo, 'pos')
labels = {nodo: str(nodo) for nodo in grafo.nodes()}
nx.draw_networkx(grafo, pos, labels=labels, with_labels=True, node_color='skyblue', edge_color='gray')
plt.title('Grafo Completo')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.grid(True)
plt.show()


# Calcular y mostrar la aptitud de la solución final
print("Distancia:", evaluar_fitness(mejor_solucion, coordenadas))
# Mostrar el recorrido en consola
print("Mejor recorrido:")
for nodo in mejor_solucion:
    print(nodo, end=' -> ')
print(mejor_solucion[0])

# Graficar el recorrido de la mejor solución
mejor_rec = mejor_solucion + [mejor_solucion[0]]  # Cerrar el ciclo
x = [coordenadas[nodo * 2] for nodo in mejor_rec]
y = [coordenadas[nodo * 2 + 1] for nodo in mejor_rec]
plt.figure(figsize=(8, 8))
plt.plot(x, y, 'bo-')
plt.title('Recorrido del Viajante de Comercio')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.show()

# Graficar fitness vs generación
generaciones = range(num_generaciones)
plt.plot(generaciones, mejores_distancias)
plt.title('Mejor Distancia por Generación')
plt.xlabel('Generación')
plt.ylabel('Mejor Distancia')
plt.show()

##Para 724 ciudades


In [10]:
# Leer coordenadas desde archivo
coordenadas = leer_coordenadas('map_city724.tsp')

# Parámetros del algoritmo genético
num_nodos = len(coordenadas) // 2
num_individuos = 724
num_generaciones = 100
num_padres = 362
probabilidad_mutacion = 0.1

In [ ]:
# Generar población inicial
def generar_poblacion_inicial(num_individuos, num_nodos):
    poblacion = []
    for _ in range(num_individuos):
        individuo = random.sample(range(num_nodos), num_nodos)
        poblacion.append(individuo)
    return poblacion

poblacion = generar_poblacion_inicial(num_individuos, num_nodos)

# Variables para guardar la evolución de la mejor distancia
mejores_distancias = []
mejor_solucion = None

# Ejecutar el algoritmo genético
for i in range(num_generaciones):
    # Calcular la distancia de la población actual
    distancias = [evaluar_fitness(individuo, coordenadas) for individuo in poblacion]
    
    # Encontrar y guardar la mejor solución actual
    mejor_solucion_actual = encontrar_mejor_solucion(poblacion, coordenadas)
    if mejor_solucion is None or evaluar_fitness(mejor_solucion_actual, coordenadas) < evaluar_fitness(mejor_solucion, coordenadas):
        mejor_solucion = mejor_solucion_actual
    
    # Guardar la mejor distancia de la generación actual
    mejores_distancias.append(evaluar_fitness(mejor_solucion, coordenadas))
    
    # Generar la siguiente generación
    poblacion = generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion, coordenadas)

# Graficar el grafo completo
grafo = nx.Graph()
for i in range(num_nodos):
    x = coordenadas[i * 2]
    y = coordenadas[i * 2 + 1]
    grafo.add_node(i, pos=(x, y))

for nodo1, nodo2 in itertools.combinations(range(num_nodos), 2):
    grafo.add_edge(nodo1, nodo2, weight=distancia_euclidiana(nodo1, nodo2, coordenadas))

pos = nx.get_node_attributes(grafo, 'pos')
labels = {nodo: str(nodo) for nodo in grafo.nodes()}
nx.draw_networkx(grafo, pos, labels=labels, with_labels=True, node_color='skyblue', edge_color='gray')
plt.title('Grafo Completo')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.grid(True)
plt.show()

# Calcular y mostrar la aptitud de la solución final
print("Distancia:", evaluar_fitness(mejor_solucion, coordenadas))
# Mostrar el recorrido en consola
print("Mejor recorrido:")
for nodo in mejor_solucion:
    print(nodo, end=' -> ')
print(mejor_solucion[0])

# Graficar el recorrido de la mejor solución
mejor_rec = mejor_solucion + [mejor_solucion[0]]  # Cerrar el ciclo
x = [coordenadas[nodo * 2] for nodo in mejor_rec]
y = [coordenadas[nodo * 2 + 1] for nodo in mejor_rec]
plt.figure(figsize=(8, 8))
plt.plot(x, y, 'bo-')
plt.title('Recorrido del Viajante de Comercio')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.show()

# Graficar fitness vs generación
generaciones = range(num_generaciones)
plt.plot(generaciones, mejores_distancias)
plt.title('Mejor Distancia por Generación')
plt.xlabel('Generación')
plt.ylabel('Mejor Distancia')
plt.show()

##Para 1173 ciudades


In [14]:
# Leer coordenadas desde archivo
coordenadas = leer_coordenadas('map_city1173.tsp')

# Parámetros del algoritmo genético
num_nodos = len(coordenadas) // 2
num_individuos = 1173
num_generaciones = 10
num_padres = 293
probabilidad_mutacion = 0.1

In [ ]:
# Generar población inicial
def generar_poblacion_inicial(num_individuos, num_nodos):
    poblacion = []
    for _ in range(num_individuos):
        individuo = random.sample(range(num_nodos), num_nodos)
        poblacion.append(individuo)
    return poblacion

poblacion = generar_poblacion_inicial(num_individuos, num_nodos)

# Variables para guardar la evolución de la mejor distancia
mejores_distancias = []
mejor_solucion = None

# Ejecutar el algoritmo genético
for i in range(num_generaciones):
    # Calcular la distancia de la población actual
    distancias = [evaluar_fitness(individuo, coordenadas) for individuo in poblacion]
    
    # Encontrar y guardar la mejor solución actual
    mejor_solucion_actual = encontrar_mejor_solucion(poblacion, coordenadas)
    if mejor_solucion is None or evaluar_fitness(mejor_solucion_actual, coordenadas) < evaluar_fitness(mejor_solucion, coordenadas):
        mejor_solucion = mejor_solucion_actual
    
    # Guardar la mejor distancia de la generación actual
    mejores_distancias.append(evaluar_fitness(mejor_solucion, coordenadas))
    
    # Generar la siguiente generación
    poblacion = generar_nueva_generacion(poblacion, num_padres, probabilidad_mutacion, coordenadas)

# Graficar el grafo completo
grafo = nx.Graph()
for i in range(num_nodos):
    x = coordenadas[i * 2]
    y = coordenadas[i * 2 + 1]
    grafo.add_node(i, pos=(x, y))

for nodo1, nodo2 in itertools.combinations(range(num_nodos), 2):
    grafo.add_edge(nodo1, nodo2, weight=distancia_euclidiana(nodo1, nodo2, coordenadas))

pos = nx.get_node_attributes(grafo, 'pos')
labels = {nodo: str(nodo) for nodo in grafo.nodes()}
nx.draw_networkx(grafo, pos, labels=labels, with_labels=True, node_color='skyblue', edge_color='gray')
plt.title('Grafo Completo')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.grid(True)
plt.show()

# Calcular y mostrar la aptitud de la solución final
print("Distancia:", evaluar_fitness(mejor_solucion, coordenadas))
# Mostrar el recorrido en consola
print("Mejor recorrido:")
for nodo in mejor_solucion:
    print(nodo, end=' -> ')
print(mejor_solucion[0])

# Graficar el recorrido de la mejor solución
mejor_rec = mejor_solucion + [mejor_solucion[0]]  # Cerrar el ciclo
x = [coordenadas[nodo * 2] for nodo in mejor_rec]
y = [coordenadas[nodo * 2 + 1] for nodo in mejor_rec]
plt.figure(figsize=(8, 8))
plt.plot(x, y, 'bo-')
plt.title('Recorrido del Viajante de Comercio')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.show()

# Graficar fitness vs generación
generaciones = range(num_generaciones)
plt.plot(generaciones, mejores_distancias)
plt.title('Mejor Distancia por Generación')
plt.xlabel('Generación')
plt.ylabel('Mejor Distancia')
plt.show()